<h1 style="text-align: center; vertical-align: middle;">Numerical Methods in Accelerator Physics</h1>
<h2 style="text-align: center; vertical-align: middle;">Lecture Series by Dr. Adrian Oeftiger</h2>

<div style="width: 45%; margin: auto; margin-top: 1em; vertical-align: middle;">
<img src="./img/etit.png" style="width: 30%; float: left; margin-right: 5%;" /><img src="./img/GSI_Logo.png" style="width: 35%; float: left; margin-right: 5%; padding-top: 0.5em;" /><img src="./img/FAIR_Logo.png" style="width: 25%; float: left;" />
</div>

<h3 style="clear: both; text-align: center; margin-top: 1em;">Lecture 3</h3>

<h2>Run this notebook online!</h2>

Interact and run this jupyter notebook online:

<div class="alert alert-block alert-info" style="text-align:center;">
via the public mybinder.org service: <br />
<a href="https://mybinder.org/v2/gh/aoeftiger/TUDa-NMAP-2023/v3.0" style="width:auto; display:table;margin:0.5em auto;"><img src="./img/binder_logo.svg" alt="mybinder.org logo" height="1ex" /></a>
</div>

<div class="alert alert-block alert-info" style="text-align:center;">
via the public gesis.org service: <br />
<a href="https://notebooks.gesis.org/binder/v2/gh/aoeftiger/TUDA-NMAP-2023/v3.0" style="width:auto; display:table;margin:0.5em auto;"><img src="./img/binder_logo.svg" alt="gesis.org logo" height="1ex" /></a>
</div>

Also find this lecture rendered [as HTML slides on github $\nearrow$](https://aoeftiger.github.io/TUDa-NMAP-2023/) along with the [source repository $\nearrow$](https://github.com/aoeftiger/TUDa-NMAP-2023).

<h2>Run this first!</h2>

Imports and modules:

In [ ]:
from config import (np, plt, hamiltonian, 
                    plot_hamiltonian, solve_leapfrog, dt)
import PyNAFF
%matplotlib inline

<h2 style="color: #b51f2a">Refresher!</h2>

- coordinate system: $\zeta=(x,x',y,y',z,\delta)$
- rms <b>emittance</b> (phase-space area, thermal energy)
- non-linearities and <b>filamentation</b>: microscopic level (Liouville theorem) vs. macroscopic level (emittance growth)
- <b>emittance preservation</b> and injection <b>mismatch</b>
- <b>modelling error</b> (vs. discretisation error)
- discrete frequency analysis:
    - <b>FFT</b>: Fast Fourier Transform
    - <b>NAFF</b>: Numerical Analysis of Fundamental Freqencies

<h2 style="color: #b51f2a">Today!</h2>

1. Chaos and Early Indicators
2. Numerical artefacts: rounding and truncation

<h2>Non-linearities</h2>

Non-linearities in particle dynamics can originate from e.g.:

- <b>stray fields</b> in drift spaces
- higher-order <b>multipole components</b> in dipole magnets (steering) and quadrupole magnets (focusing)
- higher-order <b>multipole magnets</b> (sextupoles, octupoles) used to control various properties of the beam
- effects of <b>beam fields</b> on individual particles within the beam (space-charge forces, beam-beam effects when colliding two beams)

Effects can be varied and quite dramatic! $\implies$ require understanding of nonlinear dynamics to optimise design and operation of many accelerator systems!

<h2>Effects of Non-linearities</h2>

Design accelerator based on linear quasi-periodic oscillations $\implies$ focusing around reference particle, phase-space trajectory should be an ellipse!

Non-linearities in beam lines can have several effects:
- shape of phase-space ellipse can become distorted
- phase advance per period (frequency) can depend on oscillation amplitude
- motion can be stable for small amplitude but unstable at large amplitude: chaotic motion
- features such as "phase-space islands" can appear in phase-space portrait

<h2>Chaotic Dynamics</h2>

<img src="./img/double_pendulum.gif" alt="Double pendulum with slightly different initial conditions" style="width:20%; float:right; margin-left: 1em;" />

<b>Tentative definition</b>:
    
- sensitive to initial conditions: <i>butterfly effect</i>
- qualitatively recurring

but chaos is <b>not</b>:

- (strictly) periodic or <b>regular</b>
- "escalating"

<p style="clear: both; font-size: 10pt; text-align: right; float: right;"><a href="https://commons.wikimedia.org/wiki/File:Demonstrating_Chaos_with_a_Double_Pendulum.gif">image by Ari Rubinsztejn</a></p>

<div style="text-align: center; margin:auto;">
<h3>... what is the problem?</h3>

$\implies$ motion <b>not predictable</b> despite of deterministic dynamics! (correct initial condition? numerical precision? ...)

$\implies$ cannot exclude sudden changes of amplitude on long-term time scales (synchrotron storage times) from <b>short-term simulations</b> in case of chaotic motion!
</div>

<h2>Deterministic Chaos</h2>

<b>Necessary condition</b>: non-linearity

<i>"The exponential divergence or convergence of nearby trajectories (Lyapunov exponents) is conceptually the most basic indicator of deterministic chaos."</i>
<p style="text-align: right;">M. Sano and Y. Sawada (1985) <br />Phys. Rev. Lett. <b>55</b>, 1082</p>

<b>One possible conceptual approach</b>: 
<div class="alert alert-block alert-success" style="text-align:center;">deterministic chaos $\doteq$ "bounded, deterministic dynamics with a positive Lyapunov exponent."</div>


<h2>(Maximum) Lyapunov Exponent</h2>

Consider two nearby trajectories $\zeta_1, \zeta_2$ evolving over path length $s$:

- infinitesimal distance $|\zeta_1-\zeta_2|\rightarrow 0$
- infinite evolution $s\rightarrow \infty$

Chaotic behaviour if distance grows or shrinks exponentially:

$$|\zeta_1(s_0 + s) - \zeta_2(s_0 + s)| = \underbrace{|\zeta_1(s_0) - \zeta_2(s_0)|}\limits_{\Delta\zeta(s_0)} \cdot e^{\lambda_1 s}$$

$\implies$ $\lambda_1$: maximum <b>Lyapunov exponent</b>

<div class="alert alert-block alert-success" style="text-align:center;">$$ \lambda_1 \doteq \lim\limits_{s\rightarrow \infty}\,\lim\limits_{\Delta\zeta(s_0)\rightarrow 0}\,\frac{1}{s}\,\ln\left(\frac{\Delta\zeta(s_0+s)}{\Delta\zeta(s_0)}\right)$$</div>

<h2>Lyapunov Spectrum</h2>

When chaotic: perturbation aligns along direction of strongest expansion (or weakest contraction), associated with maximum Lyupunov exponent 

$\implies$ orthogonal directions for further Lyapunov exponents $\lambda_i$

$\implies$ as many $\lambda_i$ as phase-space coordinates

<h2>Example: Lorenz Attractor</h2>

Studied by E. Lorenz in 1963 for atmospheric convection:

<img src="./img/Lorenz_attractor.gif" alt="Lorenz attractor" style="width:20%; float:right; margin-left:1em;" />


$$\begin{cases}
    \cfrac{dx}{dt}=\sigma(y-x) \\
    \cfrac{dy}{dt}=x(\rho - z) - y \\
    \cfrac{dz}{dt}=xy - \beta z
\end{cases}$$
    
Lorenz used $\sigma=10, \beta=\frac{8}{3}, \rho=28$ and investigated chaotic motion.

<p style="clear:both; font-size:10pt; text-align: right; float:right;"><a href="https://commons.wikimedia.org/wiki/File:A_Trajectory_Through_Phase_Space_in_a_Lorenz_Attractor.gif">image by Dan Quinn</a></p>

<h2>Chaotic Motion in Lorenz Attractor</h2>

Two identical Lorenz oscillators with initial conditions.
Second oscillator is slightly perturbed ($10^{-14}$) at $t = 30$:
    
<img src="./img/Lorenz_Lyapunov.png" style="width:70%; margin:auto;" alt="Trajectories in Lorenz attractor" />

<p style="font-size: 10pt; text-align: right; "><a href="https://www.ukbonn.de/site/assets/files/22947/02-wt-dynsystchaos.pdf">image by G. Ansmann</a></p>

<h2>Returning to Discrete Pendulum</h2>

In [ ]:
N = 11

thetas = np.linspace(0, 0.99 * np.pi, 11)
ps = np.zeros_like(thetas)

In [ ]:
plt.scatter(thetas, ps, c='b', marker='.')

plot_hamiltonian()

<h2>Time evolution</h2>

Numerical integration of equations of motion for distribution of pendulums, using leapfrog (drift+kick+drift):

In [ ]:
n_steps = 1000

In [ ]:
results_thetas = np.zeros((n_steps, N), dtype=np.float32)
results_thetas[0] = thetas

results_ps = np.zeros((n_steps, N), dtype=np.float32)
results_ps[0] = ps

for k in range(1, n_steps):
    results_thetas[k], results_ps[k] = solve_leapfrog(results_thetas[k - 1], results_ps[k - 1])

In [ ]:
plt.scatter(results_thetas, results_ps, c='b', marker='.', s=1)

plot_hamiltonian()

<h2>Chaos near Unstable Fixed Point</h2>

Let us investigate the unbounded, continuously rotating pendulum, with an energy just above the separatrix value:

In [ ]:
N = 2

thetas = np.pi * np.ones(N, dtype=np.float32)
ps = np.linspace(0.01, 0.05, N)

In [ ]:
results_thetas2 = np.zeros((n_steps, N), dtype=np.float32)
results_thetas2[0] = thetas

results_ps2 = np.zeros((n_steps, N), dtype=np.float32)
results_ps2[0] = ps

for k in range(1, n_steps):
    results_thetas2[k], results_ps2[k] = solve_leapfrog(results_thetas2[k - 1], results_ps2[k - 1])

We plot subsequent passages with the ever increasing $\theta$ projected to the interval $\bigl[0, 2\pi\bigr)$ :

In [ ]:
plt.scatter(results_thetas2 % (2 * np.pi), results_ps2, c='b', marker='.', s=1)
plt.scatter([np.pi], [0], c='r', marker='o')

plt.xlim(np.pi - 0.1, np.pi + 0.1)
plt.ylim(-0.01, 0.1)
plt.xlabel(r'$\theta$')
plt.ylabel(r'$p$');

$\implies$ Observation near the <span style="color: red;">red unstable fixed point</span>:

The phase-space trajectory becomes <b>non-regular</b>! 

$\implies$ The discrete pendulum dynamics features <b>chaotic motion</b>.

(The discrete pendulum equations are also known as the <i>Chirikov standard map</i>.)

<h2>Qualitative Investigation of Local Lyapunov Exponent</h2>

First investigate around stable fixed point $\theta=0$, $p=0$.

(Later come back here and investigate the around unstable fixed point $\theta=\pi$, $p=0$.)

In [ ]:
dist = 1e-10

thetas = 0 * np.pi * np.ones(N, dtype=np.float64) # change this line
ps = np.array([0.001, 0.001 + dist], dtype=np.float64)

In [ ]:
n_steps = 100000

In [ ]:
results_thetas3 = np.zeros((n_steps, N), dtype=np.float64)
results_thetas3[0] = thetas

results_ps3 = np.zeros((n_steps, N), dtype=np.float64)
results_ps3[0] = ps

for k in range(1, n_steps):
    results_thetas3[k], results_ps3[k] = solve_leapfrog(results_thetas3[k - 1], results_ps3[k - 1])

In [ ]:
plt.plot(results_thetas3[:, 0], label='$p_{ini}$')
plt.plot(results_thetas3[:, 1], label='$p_{ini} + \Delta p_{ini}$')

# plt.xlim(100000-1000, 100000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$')
plt.legend();

<h2>Euclidean Distance $|(\theta_2,p_2)-(\theta_1,p_1)|$</h2>

In [ ]:
results_dist = np.sqrt(
    (results_thetas3[:, 1] - results_thetas3[:, 0])**2 + 
    (results_ps3[:, 1] - results_ps3[:, 0])**2
)

In [ ]:
plt.plot(results_dist)

plt.yscale('log')
plt.xlabel('Steps $k$')
plt.ylabel('Phase-space distance');

<p style="color: #e6541a;">$\implies$ how do you interpret this picture?</p>

<p style="color: #e6541a;"><b>Exercise:</b> Go back and investigate the distance evolution for two trajectories near to the unstable fixed point!</p>

<h2>Evaluating Local Maximum Lyapunov Exponent</h2>

Nearly exponential increase over two periods (first 4000 steps), then bounded by system size.

Local Maximum Lyapunov exponent estimated by simple linear regression:

$$\lambda_1 \approx \mathrm{slope}\left(\frac{1}{k\Delta t} \ln\left(\frac{|(\theta_2,p_2)-(\theta_1,p_1)|}{10^{-10}}\right)\right)$$

In [ ]:
B, A = np.polyfit(
    x=np.arange(n_steps)[:4000],
    y=np.log(results_dist[:4000] / dist),
    deg=1
)

B

In [ ]:
plt.plot(results_dist[:4000] / dist)
plt.plot(np.exp(B * np.arange(n_steps)[:4000] + A))

plt.yscale('log');

<h2>Frequency Diffusion</h2>

Use NAFF algorithm as early indicator of chaotic motion in periodic systems:

<div class="alert alert-block alert-success" style="text-align:center;">
Regular orbits $\Longleftrightarrow$ fixed frequency up to numerical accuracy!</div>

<p style="text-align:center;width:100%;">vs.</p>

<div class="alert alert-block alert-success" style="text-align:center;">
Non-regular orbits $\Longleftrightarrow$ frequency evolves in time indicating chaotic diffusion of orbit!</div>

Further reading: "Introduction to Frequency Map Analysis" by J. Laskar in <a href="https://link.springer.com/content/pdf/10.1007/978-94-011-4673-9.pdf">Hamiltonian Systems with Three or More Degrees of Freedom, Springer (1999), pp. 134-150</a>

<h2>Example: Earth-Moon System</h2>
    
Precession of Earth is stabilised by presence of Moon:

<img src="./img/earth-moon.png" alt="Earth precession frequency vs. amplitude" style="width:60%; margin:auto;" />

<p style="clear:both; font-size:10pt; text-align: right; float:right;"><a href="https://link.springer.com/content/pdf/10.1007/978-94-011-4673-9.pdf">image by J. Laskar</a></p>

<h2>Example: CERN LHC</h2>

<img src="./img/lhc-fma.png" alt="Frequency Map Analysis of CERN LHC" style="float: right; width:40%; margin-left: 1em;" />

Frequency Map Analysis (FMA) to identify diffusion due to non-linear resonances:

<div class="alert alert-block alert-success" style="text-align:center;">Concept of <b>dynamic aperture</b>: largest amplitude where particles survive (before chaos and amplitude increase can lead to loss!)!</div>

$\implies$ always seek to maximise dynamic aperture in accelerators

$\implies$ use of "early chaos indicators" such as Lyapunov exponents or FMA: identification and mitigation/correction of sources

<p style="clear:both; font-size:10pt; text-align: right; float:right;"><a href="https://aip.scitation.org/doi/pdf/10.1063/1.4884495">image by Y. Papaphilippou</a></p>

<h2>Try Concept on Discrete Pendulum</h2>

Can we detect frequency diffusion for the discrete pendulum?

In [ ]:
n_steps = 100000

In [ ]:
theta_ini = np.pi - 0.001
p_ini = 0

In [ ]:
results_thetas4 = np.zeros(n_steps, dtype=np.float64)
results_thetas4[0] = theta_ini

results_ps4 = np.zeros(n_steps, dtype=np.float64)
results_ps4[0] = p_ini

for k in range(1, n_steps):
    results_thetas4[k], results_ps4[k] = solve_leapfrog(results_thetas4[k - 1], results_ps4[k - 1])

In [ ]:
plt.plot(results_thetas4)

plt.xlim(0, 1000)

plt.xlabel('Steps $k$')
plt.ylabel(r'$\theta$');

In [ ]:
window_length = 1000

freqs_naff = []

for signal in results_thetas4.reshape((n_steps // window_length, window_length)):
    freq_naff = PyNAFF.naff(signal - np.mean(signal), turns=window_length, nterms=1)[0, 1]
    freqs_naff += [freq_naff]

In [ ]:
plt.plot(np.arange(0, n_steps, window_length), freqs_naff, ls='none', marker='.')

plt.xlabel('Steps $k$')
plt.ylabel('NAFF determined frequency');

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(freqs_naff);

<h2>Summing up Concepts</h2>

- sources for non-linearities in accelerators
- deterministic chaos: (bounded) deterministic dynamics with a positive Lyapunov exponent
    - sensitivity to initial conditions
    - prevents prediction for long-term time scales
- early indicators of chaos:
    - (maximum) Lyapunov exponent
    - Frequency Map Analysis (FMA)

<h2>Floating Point Representation</h2>

Represent real numbers $r\in\mathbb{R}$ on computers by "floats":
- finite and discrete set of numbers
- floating-point form: $r=\underbrace{6.022}\limits_\text{significand}\times 10{\underbrace{{}^{23}}\limits_\text{exponent}}$

Todays standard: IEEE 754

$r=\pm a \times 2^{e}$

e.g. a "single-precision" float (FP32):

- 1 sign bit
- $t=23$ bits for significand $a$
- 8 bits for exponents in range $e\in[-126, 127]$
- $\pm\infty$ and NaN

<h2>Numerical Artefacts</h2>

Floating point representation of numbers on computers $\leadsto$ numerical artefacts:
- rounding: storing the least significant bit depending on arithmetic operation
- truncating: e.g. π cannot be stored exactly but is truncated

Smallest "machine" precision in resolving real numbers: $2^{-t}$
- FP32: $2^{-23}\approx 10^{-7}$
- FP64: $2^{-52}\approx 10^{-15}$

<div class="alert alert-block alert-warning" style="text-align:center;">Accumulation of errors $\implies$ gradual loss of significance</div>

Think about what this means for simulating long-term dynamics, e.g. for LHC storage times...

<!--<h2>Modelling the Rounding Error $\varepsilon$</h2>

For a number $k$ of computational steps:

- in the worst case can be a run-away: always leaning to the same side (e.g. always rounding up)
- in the average case: random walk with $\varepsilon\propto \sqrt{k}$
-->

<h2>(Drastic) Example of Truncation Error</h2>

The python library `mpmath` works with arbitrary numerical precision (`workdps` for decimal precision):

In [ ]:
import mpmath as mp

with mp.workdps(7):
    x = mp.mpf(10) / 9   # == 1.11...
    
    for _ in range(30):
        print (x)
        x = (x - 1) * 10

<h2>Overview: 3 Sources of Simulation Errors</h2>
    
1. Discretisation error
2. Modelling error
3. Numerical artefacts
<br />

4. (Input error...)

<h2>Summary</h2>

- sources for non-linearities in accelerators
- deterministic chaos
- early indicators:
    - (maximum) Lyapunov exponent
    - Frequency Map Analysis
- numerical artefacts: rounding and truncation, machine precision

<h2>Literature</h2>

- chapter 2.1 in Justin Solomon, [Numerical Algorithms](https://people.csail.mit.edu/jsolomon/share/book/numerical_book.pdf)